# CitiBike Insurance Partnership Analysis

**Purpose**: Usage patterns and business decision assets for AXA partnership

**Run via**: `make run-notebooks MODE=nyc` or `make all-both`

In [ ]:
# ============================================================
# SETUP
# ============================================================
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Configuration from Makefile
RUN_DIR = os.environ.get("CITIBIKE_RUN_DIR")
mode = os.environ.get("CITIBIKE_MODE", "unknown")

if not RUN_DIR:
    raise ValueError(
        "CITIBIKE_RUN_DIR not set.\n"
        "Run via Makefile: make run-notebooks MODE=nyc"
    )

RUN_DIR = Path(RUN_DIR)
run_label = RUN_DIR.name

print(f"Mode: {mode}")
print(f"Run directory: {RUN_DIR}")
print(f"Run label: {run_label}")

# Load CSVs
df_year = pd.read_csv(RUN_DIR / "citibike_trips_by_year.csv")
df_month = pd.read_csv(RUN_DIR / "citibike_trips_by_month.csv")
df_dow = pd.read_csv(RUN_DIR / "citibike_trips_by_dow.csv")
df_hour = pd.read_csv(RUN_DIR / "citibike_trips_by_hour.csv")

# Scorecard (try 500m first)
scorecard_path = RUN_DIR / "axa_partner_scorecard_500m.csv"
if not scorecard_path.exists():
    for p in RUN_DIR.glob("axa_partner_scorecard_*m.csv"):
        scorecard_path = p
        break
df_score = pd.read_csv(scorecard_path) if scorecard_path.exists() else None

# Windows
windows_path = RUN_DIR / "axa_target_windows.csv"
df_windows = pd.read_csv(windows_path) if windows_path.exists() else None

# Highlights path
highlights_path = RUN_DIR / "summary_highlights.md"

print(f"\nLoaded:")
print(f"  df_year:    {len(df_year)} rows")
print(f"  df_month:   {len(df_month)} rows")
print(f"  df_dow:     {len(df_dow)} rows")
print(f"  df_hour:    {len(df_hour)} rows")
print(f"  df_score:   {len(df_score) if df_score is not None else 'None'} rows")
print(f"  df_windows: {len(df_windows) if df_windows is not None else 'None'} rows")

# Figure output
FIG_DIR = RUN_DIR.parent.parent / "reports" / run_label / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

def savefig(name):
    path = FIG_DIR / name
    plt.savefig(path, dpi=150, bbox_inches="tight")
    print(f"Saved: {path}")

In [ ]:
# --- Quick text highlights from summarize script (if present) ---
if highlights_path.exists():
    txt = highlights_path.read_text(encoding="utf-8", errors="ignore")
    display(Markdown(txt))
else:
    print("No summary_highlights.md found at:", highlights_path)

In [ ]:
# --- Trips by year (THIS RUN) ---
g = df_year.copy()
g["year"] = pd.to_numeric(g["year"], errors="coerce")
g = g.dropna(subset=["year"]).sort_values("year").reset_index(drop=True)

display(g)

# Plot
plt.figure(figsize=(10, 6))

years = [int(y) for y in g["year"]]
trips = [int(t) for t in g["trips"]]

plt.plot(years, trips, marker="o", linewidth=2, markersize=8, color='#2E86AB')
plt.xticks(years, [str(y) for y in years])
plt.grid(True, alpha=0.3, linestyle='--')

for year, trip in zip(years, trips):
    plt.text(year, trip, f'{trip/1e6:.2f}M', ha='center', va='bottom', fontsize=9)

plt.title(f"Trips by year — mode={mode} ({run_label})", fontsize=14, fontweight='bold')
plt.xlabel("Year", fontsize=12)
plt.ylabel("Trips", fontsize=12)
plt.tight_layout()

savefig("01_trips_by_year.png")
plt.show()

In [ ]:
# --- Trips by month (THIS RUN) ---
m = df_month.copy()
m["month"] = pd.to_numeric(m["month"], errors="coerce").astype("Int64")
m["year"] = pd.to_numeric(m["year"], errors="coerce").astype("Int64")

display(m.sort_values(["year", "month"]))

# Plot
m2 = m.dropna(subset=["year", "month"]).copy()
m2["ym"] = m2["year"].astype(int).astype(str) + "-" + m2["month"].astype(int).astype(str).str.zfill(2)

plt.figure(figsize=(10, 4))
plt.bar(m2["ym"], m2["trips"])
plt.title(f"Trips by year-month — mode={mode} ({run_label})")
plt.xlabel("Year-Month")
plt.ylabel("Trips")
plt.xticks(rotation=75, ha="right")
plt.tight_layout()

savefig("02_trips_by_year_month.png")
plt.show()

In [ ]:
# --- Month-of-year seasonality (one line per year) ---
m = df_month.copy()
m["month"] = pd.to_numeric(m["month"], errors="coerce")
m["year"] = pd.to_numeric(m["year"], errors="coerce")
m = m.dropna(subset=["year", "month"])

plt.figure(figsize=(10, 5))

for yr in sorted(m["year"].unique()):
    sub = m[m["year"] == yr].sort_values("month")
    plt.plot(sub["month"], sub["trips"], marker="o", label=str(int(yr)))

plt.title(f"Month-of-year seasonality — mode={mode} ({run_label})", fontsize=14, fontweight='bold')
plt.xlabel("Month", fontsize=12)
plt.ylabel("Trips", fontsize=12)
plt.xticks(range(1, 13))
plt.legend(title="Year")
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()

savefig("03_month_of_year_seasonality.png")
plt.show()

In [ ]:
# --- Day-of-week patterns ---
d = df_dow.copy()
d["dow"] = pd.to_numeric(d["dow"], errors="coerce")
d["trips"] = pd.to_numeric(d["trips"], errors="coerce").fillna(0)
d["year"] = pd.to_numeric(d["year"], errors="coerce")
d = d.dropna(subset=["dow", "year"]).copy()
d["dow"] = d["dow"].astype(int)
d["year"] = d["year"].astype(int)

dow_map = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday",
           4: "Friday", 5: "Saturday", 6: "Sunday"}
d["dow_name"] = d["dow"].map(dow_map)

display(d.sort_values(["year", "dow"]))

# Pivot for plotting
pivot = d.pivot_table(index=["dow", "dow_name"], columns="year", values="trips", aggfunc="sum").sort_index(level=0)

plt.figure(figsize=(10, 4))
x_labels = [n for n in pivot.index.get_level_values("dow_name")]
x = np.arange(len(x_labels))

for yr in pivot.columns:
    plt.plot(x, pivot[yr].values, marker="o", label=str(yr))

plt.xticks(x, x_labels, rotation=30, ha="right")
plt.title(f"Day-of-week pattern — mode={mode} ({run_label})", fontsize=14, fontweight='bold')
plt.xlabel("Day of Week", fontsize=12)
plt.ylabel("Trips", fontsize=12)
plt.legend(title="Year")
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()

savefig("04_dow_pattern.png")
plt.show()

In [ ]:
# --- Day-of-week % shares per year ---
shares = d.groupby(["year", "dow", "dow_name"], as_index=False)["trips"].sum().sort_values(["year", "dow"])
shares["pct_of_year"] = shares["trips"] / shares.groupby("year")["trips"].transform("sum") * 100.0

display(shares)

pivot_pct = shares.pivot(index="dow_name", columns="year", values="pct_of_year")
pivot_pct = pivot_pct.reindex([dow_map[i] for i in range(7)])

plt.figure(figsize=(10, 4))
x = np.arange(len(pivot_pct.index))

for yr in pivot_pct.columns:
    plt.plot(x, pivot_pct[yr].values, marker="o", label=str(yr))

plt.xticks(x, pivot_pct.index, rotation=30, ha="right")
plt.title(f"Day-of-week % share — mode={mode} ({run_label})", fontsize=14, fontweight='bold')
plt.xlabel("Day of Week", fontsize=12)
plt.ylabel("% of Year's Trips", fontsize=12)
plt.legend(title="Year")
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()

savefig("05_dow_pct_share.png")
plt.show()

In [ ]:
# --- Hour-of-day patterns ---
h = df_hour.copy()
h["hour"] = pd.to_numeric(h["hour"], errors="coerce")
h["trips"] = pd.to_numeric(h["trips"], errors="coerce").fillna(0)
h["year"] = pd.to_numeric(h["year"], errors="coerce")
h = h.dropna(subset=["hour", "year"]).copy()

display(h.sort_values(["year", "hour"]).head(20))

# Check for week_part column
part_col = "week_part" if "week_part" in h.columns else None

plt.figure(figsize=(12, 5))

if part_col:
    for (yr, seg), sub in h.groupby(["year", part_col]):
        sub = sub.sort_values("hour")
        plt.plot(sub["hour"], sub["trips"], marker="o", label=f"{int(yr)} | {seg}")
else:
    for yr, sub in h.groupby("year"):
        sub = sub.sort_values("hour")
        plt.plot(sub["hour"], sub["trips"], marker="o", label=str(int(yr)))

plt.title(f"Hour-of-day pattern — mode={mode} ({run_label})", fontsize=14, fontweight='bold')
plt.xlabel("Hour", fontsize=12)
plt.ylabel("Trips", fontsize=12)
plt.xticks(range(0, 24))
plt.legend(title="Year / Segment", fontsize=8, ncol=2)
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()

savefig("06_hour_of_day.png")
plt.show()

In [ ]:
# --- Crash proximity risk proxy (NYC only): YEARLY comparison ---
MIN_TRIPS_FOR_CREDIBLE_DISPLAY = 5000
TOP_N = 20

if df_score is None or df_score.empty:
    print("Skipping yearly risk analysis - no scorecard data available")
elif "year" not in df_score.columns:
    print("Scorecard has no 'year' column (overall aggregate only).")
else:
    r = df_score.copy()
    
    # Normalize column names
    r = r.rename(columns={
        "start_station_id": "station_id",
        "start_station_name": "station_name"
    }, errors="ignore")
    
    # Get trips column
    r["trips"] = pd.to_numeric(r.get("exposure_trips", r.get("trips", 0)), errors="coerce")
    r = r[r["trips"] >= MIN_TRIPS_FOR_CREDIBLE_DISPLAY].copy()
    
    years = sorted(r["year"].dropna().unique())
    
    print(f"\n=== Yearly Risk Analysis (using scorecard EB estimates) ===")
    print(f"Stations with ≥{MIN_TRIPS_FOR_CREDIBLE_DISPLAY:,} trips")
    
    yearly_burden = []
    
    for yy in years:
        ry = r[r["year"] == yy].copy()
        if ry.empty:
            continue
        
        print(f"\n--- Year {int(yy)} ---")
        
        print(f"Top {TOP_N} by exposure:")
        display(ry.sort_values("trips", ascending=False).head(TOP_N)[
            ["mode", "year", "station_id", "station_name", "trips"]
        ])
        
        print(f"Top {TOP_N} by EB risk rate:")
        display(ry.sort_values("eb_risk_rate_per_100k_trips", ascending=False).head(TOP_N)[
            ["mode", "year", "station_id", "station_name", "trips", 
             "crash_count", "eb_risk_rate_per_100k_trips"]
        ])
        
        if "expected_incidents_proxy" in ry.columns:
            yearly_burden.append({"year": int(yy), "eb_expected_crashes": ry["expected_incidents_proxy"].sum()})
    
    # Plot yearly burden
    if yearly_burden:
        yb = pd.DataFrame(yearly_burden).sort_values("year")
        
        plt.figure(figsize=(9, 4))
        plt.bar(yb["year"].astype(str), yb["eb_expected_crashes"].values)
        plt.title(f"Yearly EB crash-proxy burden — mode={mode}")
        plt.xlabel("Year")
        plt.ylabel("Expected incidents (sum across stations)")
        plt.tight_layout()
        savefig("09_yearly_EB_crash_proxy_burden.png")
        plt.show()
        
        print(f"\nTotal EB expected crashes by year:")
        display(yb)

In [ ]:
# --- Exposure vs Risk "Zones" (EB-smoothed quadrant view) ---
MIN_TRIPS_FOR_ZONES = 5000

if df_score is None or df_score.empty:
    print("No scorecard data — skipping zones plot")
else:
    r = df_score.copy()
    
    # Normalize column names
    r = r.rename(columns={
        "start_station_id": "station_id",
        "start_station_name": "station_name",
        "exposure_trips": "trips"
    }, errors="ignore")
    
    # Get risk metric
    if "eb_risk_rate_per_100k_trips" in r.columns:
        r["risk_rate"] = pd.to_numeric(r["eb_risk_rate_per_100k_trips"], errors="coerce")
        risk_metric_name = "EB-smoothed risk rate per 100k trips"
    elif "risk_rate_per_100k_trips" in r.columns:
        r["risk_rate"] = pd.to_numeric(r["risk_rate_per_100k_trips"], errors="coerce")
        risk_metric_name = "Raw risk rate per 100k trips"
    else:
        print("No risk rate column found")
        r = None
    
    if r is not None:
        r["trips"] = pd.to_numeric(r["trips"], errors="coerce")
        r = r.dropna(subset=["trips", "risk_rate"])
        r = r[r["trips"] >= MIN_TRIPS_FOR_ZONES].copy()
        
        if len(r) == 0:
            print(f"No stations meet trips ≥ {MIN_TRIPS_FOR_ZONES} — skipping zones plot.")
        else:
            print(f"Plotting {len(r):,} stations with ≥ {MIN_TRIPS_FOR_ZONES:,} trips")
            
            x_med = r["trips"].median()
            y_med = r["risk_rate"].median()
            
            print(f"Median exposure: {x_med:,.0f} trips")
            print(f"Median risk: {y_med:.2f}")
            
            # Classify into zones
            def zone(row):
                hi_x = row["trips"] >= x_med
                hi_y = row["risk_rate"] >= y_med
                if hi_x and hi_y: return "High exposure / High risk"
                if hi_x: return "High exposure / Lower risk"
                if hi_y: return "Lower exposure / High risk"
                return "Lower exposure / Lower risk"
            
            r["zone"] = r.apply(zone, axis=1)
            
            print("\nZone distribution:")
            display(r["zone"].value_counts().to_frame("stations"))
            
            # Plot
            zone_colors = {
                "High exposure / High risk": "#d62728",
                "High exposure / Lower risk": "#2ca02c",
                "Lower exposure / High risk": "#ff7f0e",
                "Lower exposure / Lower risk": "#1f77b4"
            }
            
            plt.figure(figsize=(10, 7))
            
            for z, sub in r.groupby("zone"):
                plt.scatter(sub["trips"], sub["risk_rate"], alpha=0.6, s=60,
                           label=f"{z} (n={len(sub)})", color=zone_colors.get(z, "#999999"))
            
            plt.axvline(x_med, linestyle="--", color="gray", alpha=0.7, linewidth=1.5,
                       label=f"Median exposure ({x_med:,.0f})")
            plt.axhline(y_med, linestyle="--", color="gray", alpha=0.7, linewidth=1.5,
                       label=f"Median risk ({y_med:.2f})")
            
            plt.xscale("log")
            plt.xlabel("Exposure (trips, log scale)", fontsize=11)
            plt.ylabel(risk_metric_name, fontsize=11)
            plt.title(f"Exposure vs Risk Zones — EB-smoothed\n(stations with ≥ {MIN_TRIPS_FOR_ZONES:,} trips, mode={mode})",
                     fontsize=12, fontweight="bold")
            plt.legend(fontsize=8, loc="best")
            plt.grid(True, alpha=0.3, linestyle=":")
            plt.tight_layout()
            
            savefig(f"10_zones_exposure_vs_risk.png")
            plt.show()
            
            # High priority table
            hh = r[r["zone"] == "High exposure / High risk"].sort_values("risk_rate", ascending=False).head(15)
            
            print(f"\n HIGH PRIORITY STATIONS (High exposure + High risk, top 15):")
            display_cols = [c for c in ["mode", "station_id", "station_name", "trips", "risk_rate", 
                                        "crash_count", "credibility_flag"] if c in hh.columns]
            display(hh[display_cols])
            
            # Business interpretation
            print(f"\n Business Interpretation:")
            print(f"  • {len(hh)} high-priority stations → Prevention pilots & safety interventions")
            high_exp_low_risk = len(r[r["zone"] == "High exposure / Lower risk"])
            print(f"  • {high_exp_low_risk} high-exposure/low-risk stations → Product upsell targets")

In [ ]:
# --- AXA Decision Assets (INSURER-READY): WHERE + WHEN + WHAT ---
MIN_TRIPS = 5000
TOP_N = 20

def drop_bool_cols(df):
    """Remove True/False columns for presentation."""
    bool_cols = [c for c in df.columns if df[c].dtype == bool]
    for c in df.columns:
        if c not in bool_cols and df[c].dtype == object:
            unique_vals = set(df[c].dropna().unique())
            if unique_vals.issubset({True, False, 'True', 'False'}):
                bool_cols.append(c)
    return df.drop(columns=bool_cols, errors="ignore")

if df_score is None or df_windows is None:
    print("Missing inputs:")
    print(f"  df_score:   {'OK' if df_score is not None else 'MISSING'}")
    print(f"  df_windows: {'OK' if df_windows is not None else 'MISSING'}")
else:
    score = df_score.copy()
    win = df_windows.copy()
    
    # Defensive typing
    for c in ["exposure_trips", "crash_count", "risk_rate_per_100k_trips",
              "eb_risk_rate_per_100k_trips", "expected_incidents_proxy", "axa_priority_score"]:
        if c in score.columns:
            score[c] = pd.to_numeric(score[c], errors="coerce")
    
    # Ensure credibility flag
    if "credibility_flag" not in score.columns:
        score["credibility_flag"] = np.where(
            score.get("exposure_trips", 0) >= MIN_TRIPS, "credible", "insufficient_data"
        )
    
    # Column names
    id_col = "start_station_id" if "start_station_id" in score.columns else "station_id"
    name_col = "start_station_name" if "start_station_name" in score.columns else "station_name"
    
    # === WHERE (Prevention) ===
    prevention = score[score["credibility_flag"] == "credible"].copy()
    
    if "expected_incidents_proxy" in prevention.columns:
        prevention = prevention.sort_values(["expected_incidents_proxy", "exposure_trips"], ascending=False)
        rank_label = "expected burden (risk × exposure)"
    else:
        prevention = prevention.sort_values(["axa_priority_score", "exposure_trips"], ascending=False)
        rank_label = "axa_priority_score"
    
    where_cols = [c for c in [
        "mode", "year", "month", id_col, name_col,
        "exposure_trips", "crash_count", "eb_risk_rate_per_100k_trips",
        "expected_incidents_proxy", "axa_priority_score", "credibility_flag"
    ] if c and c in prevention.columns]
    
    prevention_show = drop_bool_cols(prevention[where_cols])
    
    print(f"\nWHERE (Prevention) — Top {TOP_N} credible station-periods by {rank_label}")
    print(f"Credibility rule: exposure_trips ≥ {MIN_TRIPS:,}\n")
    display(prevention_show.head(TOP_N))
    
    print(f"\nPrevention pool size: {len(prevention):,} rows (credible)")
    if "expected_incidents_proxy" in prevention.columns:
        print(f"Total expected burden (sum expected_incidents_proxy): {prevention['expected_incidents_proxy'].sum():,.1f}")
    
    # === WHERE (Reach) ===
    reach = score.sort_values("exposure_trips", ascending=False).copy()
    
    reach_cols = [c for c in [
        "mode", "year", "month", id_col, name_col, "exposure_trips", "credibility_flag"
    ] if c and c in reach.columns]
    
    reach_show = drop_bool_cols(reach[reach_cols])
    
    print(f"\nWHERE (Reach) — Top {TOP_N} station-periods by exposure_trips")
    display(reach_show.head(TOP_N))
    
    # === WHEN ===
    for c in ["trips", "pct_of_mode_year_trips", "pct_within_week_part", "priority_metric"]:
        if c in win.columns:
            win[c] = pd.to_numeric(win[c], errors="coerce")
    
    w_rank = "priority_metric" if "priority_metric" in win.columns else "trips"
    when = win.sort_values(w_rank, ascending=False).head(TOP_N) if w_rank else win.head(TOP_N)
    
    when_cols = [c for c in [
        "window_type", "segment", "window_label",
        "trips", "pct_of_mode_year_trips", "priority_metric", "recommended_action"
    ] if c in when.columns]
    
    when_show = drop_bool_cols(when[when_cols])
    
    print(f"\nWHEN — Top {TOP_N} activation windows by {w_rank}")
    display(when_show)
    
    # === WHAT ===
    print("\nWHAT (insurer-ready plan):")
    print(f"- Prevention: prioritize credible stations (≥{MIN_TRIPS:,} trips) ranked by expected burden (risk × exposure).")
    print(f"- Timing: activate in the highest-traffic windows (ranked by {w_rank}) to maximize reach and conversion.")
    print(f"- Measurement: short-term conversion lift; mid-term change in incident proxy at treated stations; long-term claims frequency.")

In [ ]:
# --- Summary ---
print("="*60)
print(f"SUMMARY — mode={mode} ({run_label})")
print("="*60)
print(f"Total trips: {df_year['trips'].sum():,.0f}")

if df_score is not None:
    id_col = "start_station_id" if "start_station_id" in df_score.columns else "station_id"
    print(f"Unique stations: {df_score[id_col].nunique():,}")
    print(f"Station-periods: {len(df_score):,}")
    
    if "credibility_flag" in df_score.columns:
        print(f"Credible station-periods: {(df_score['credibility_flag'] == 'credible').sum():,}")
    
    if "prevention_hotspot" in df_score.columns:
        print(f"Prevention hotspots: {df_score['prevention_hotspot'].sum():,}")

print("="*60)